# creation de graphiques d'analyses des résultats post-traitements

In [75]:
import geopandas as gp
import altair as alt

from MMM import import_fichiers_mmm

## 1. fichiers sources

In [76]:
resultatsLineaires=r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\affectationComplete.shp'
cle_mmm_rhv=import_fichiers_mmm(r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\MMM\MMM_Maj_EDA2017\simplifie\MMM_simple_2019.shp',
                                            r'D:\Boulot\AffairesEnCours\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\appariementV0.shp')[1]

## 2. Comptages  
Analyse des points de comptages : calcul de la longueur des tronçons RHV supportant un type_cpt "permanent" ou "ponctuel", par classe de RHV, par rapport à la longueur totale des tronçons de cette classe

In [3]:
#import des résultats filaires
res=gp.read_file(resultatsLineaires)

In [42]:
#preparation des données
res=res.loc[~res.cat_rhv.isna()].copy()
res['long']=res.geometry.length/1000
res['cat_rhv_simple']=res.cat_rhv.apply(lambda x : x[-1])
res['grp_indic']=res.apply(lambda x : 'comptage' if x['type_cpt'] in ('permanent','ponctuel') else 'autre', axis=1)

In [72]:
#analyse du pourcentage de longueur ayant fait l'objet de comptage
baseComptageParCatRhv=alt.Chart(res.groupby(['cat_rhv_simple','grp_indic']).long.sum().reset_index()).mark_bar().encode(
    x=alt.X('cat_rhv_simple', title='Catégorie RHV simplifiée'),
    color=alt.Color('grp_indic',title='Source des trafics'))
chartComptageParCatRhv=baseComptageParCatRhv.encode(y=alt.Y('long', title='longueur (km)')) | baseComptageParCatRhv.encode(
    y=alt.Y('long', title='longueur (km)', stack='normalize', axis=alt.Axis(format='%')))
chartComptageParCatRhv

alt.HConcatChart(...)

## 3.MMM  
Comparaison des données de trafics des voies du MMM avec celle issues des points de comptages, par catégorie du RHV.

In [77]:
cle_mmm_rhv

,_uid_,NO,ident,geometry
0,1.0,2.145034e+09,46028,"LINESTRING Z (397929.900 6422226.850 0.000, 39..."
1,2.0,2.145034e+09,46027,"LINESTRING Z (399548.400 6422375.300 0.000, 39..."
2,3.0,5.908285e+08,46037,"LINESTRING Z (399548.400 6422375.300 0.000, 39..."
3,4.0,5.908285e+08,46026,"LINESTRING Z (399586.250 6422379.300 0.000, 39..."
4,5.0,5.908285e+08,46038,"LINESTRING Z (399572.670 6422359.160 0.000, 39..."
...,...,...,...,...
13174,NaN,2.145033e+09,40542,"LINESTRING Z (421649.900 6431510.010 0.000, 42..."
13175,NaN,NaN,52226,"LINESTRING Z (424980.920 6431327.760 0.000, 42..."
13176,NaN,NaN,52224,"LINESTRING Z (424897.800 6431354.460 0.000, 42..."
13177,NaN,5.790505e+07,48533,"LINESTRING Z (420627.250 6421566.590 0.000, 42..."


In [79]:
cle_mmm_rhv.merge(res.loc[res.grp_indic=='comptage']

,id,ident,domanial,groupe,cat_dig,cat_rhv,passage,rggraph_nd,rggraph_na,rgraph_dbl,...,tmjo_2_sen,type_cpt,id_cpt_exp,source,target,tmja_calcu,geometry,long,cat_rhv_simple,grp_indic
0,0,8146,4,1.0,5,4,None,11613.0,11319.0,0.0,...,1370.0,ponctuel,"72, 73",692,693,None,"LINESTRING Z (417591.040 6424518.690 0.000, 41...",0.133918,4,comptage
